In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_spending_by_mcg = '''SELECT time_period_value, mcc, spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value != '2025Q1'  
and merchant_channel = 'Online'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS'
and mcc != 'All'
and destination_country != 'UNITED KINGDOM'
GROUP BY mcc, time_period_value, spend
ORDER BY time_period_value, spend DESC'''

yearly_mcc = bq.read_bq_table_sql(client, UK_spending_by_mcg)

yearly_mcc['year'] = yearly_mcc['time_period_value'].str[:4]

yearly_mcc.head()

In [ ]:
yearly_mcc.to_csv('yearly_mcc.csv')
yearly_mcc_2 = pd.read_csv('yearly_mcc.csv')
latest_year_mcc = yearly_mcc_2[yearly_mcc_2['year'] == 2024][['mcc', 'spend']]
latest_year_mcc.to_csv('latest_year_mcc.csv', index=False)
total_spend_per_mcc = latest_year_mcc.groupby('mcc')['spend'].sum()
total_spend_per_mcc_perc = (total_spend_per_mcc / total_spend_per_mcc.sum()) * 100
total_spend_per_mcc_perc.to_csv('total_spend_per_mcc_perc.csv')

In [ ]:
df_perc = pd.read_csv('total_spend_per_mcc_perc.csv')
df_perc.columns = ['mcc', 'percentage']
top_6_mccs = df_perc.sort_values(by='percentage', ascending=False).head(6)
top_6_mccs

In [ ]:
top_6_sum = top_6_mccs.sum()
top_6_sum